In [4]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix
import numpy as np

import keras_metrics
import keras


In [ ]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = valid_datagen.flow_from_directory(
        'dataset/test',
        target_size=(150, 150),
        color_mode= "grayscale",
        batch_size=32,
        class_mode='categorical')
train_generator = train_datagen.flow_from_directory(
        'dataset/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        color_mode= "grayscale",
        batch_size=32,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
valid_generator = valid_datagen.flow_from_directory(
        'dataset/validate',
        target_size=(150, 150),
        color_mode= "grayscale",
        batch_size=32,
        class_mode='categorical')


In [ ]:
# Function to create model, required for KerasClassifier


model = Sequential()
model.add(Flatten(input_shape=(150,150)))
model.add(Dense(150, input_dim=22500, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy",keras.metrics.Precision(), keras.metrics.Recall()])
model.summary()

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=800 // 32,
        epochs=500,
        validation_data=valid_generator,
        validation_steps=200 // 32)
model.save_weights('first_try.h5') 

In [ ]:
filenames = test_generator.filenames
nb_samples = len(filenames)
pred = model.predict_generator(test_generator)#,steps = nb_samples)

In [ ]:
val_preds = np.argmax(pred, axis=-1)
val_trues = test_generator.classes
cm = confusion_matrix(val_trues, val_preds)
cm
print(val_trues)
print(val_preds)

In [ ]:
labels = test_generator.class_indices.keys()
labels

In [ ]:
precisions, recall, f1_score, _ = precision_recall_fscore_support(val_trues, val_preds)

In [ ]:
recall

In [ ]:
precisions

In [6]:
import numpy as np
import cv2
  
     
# Read the query image as query_img
# and train image This query image
# is what you need to find in train image
# Save it in the same directory
# with the name image.jpg 
train_img = cv2.imread('dataset/train/00010/00530_00000.ppm')
query_img = cv2.imread('dataset/test/00004/00719_00002.ppm')
  
trainKeypoints, m = orb.detectAndCompute(query_img,None)

# print(trainDescriptors.shape)
print("query descriptors" , m.shape)
    
# Convert it to grayscale
query_img_bw = cv2.cvtColor(query_img,cv2.COLOR_BGR2GRAY)
train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)
  
# Initialize the ORB detector algorithm
orb = cv2.ORB_create()
  
# Now detect the keypoints and compute
# the descriptors for the query image
# and train image
queryKeypoints, queryDescriptors = orb.detectAndCompute(query_img_bw,None)
trainKeypoints, trainDescriptors = orb.detectAndCompute(train_img_bw,None)

 
# Initialize the Matcher for matching
# the keypoints and then match the
# keypoints
matcher = cv2.BFMatcher()
matches = matcher.match(queryDescriptors,trainDescriptors)
  
# draw the matches to the final image
# containing both the images the drawMatches()
# function takes both images and keypoints
# and outputs the matched query image with
# its train image
final_img = cv2.drawMatches(query_img, queryKeypoints,
train_img, trainKeypoints, matches[:20],None)
  
final_img = cv2.resize(final_img, (1000,650))
 
# Show the final image
cv2.imshow("Matches", final_img)
cv2.waitKey(3000)

NameError: name 'orb' is not defined

In [5]:
import os
trainKeyposints = []
trainDescriptors = []
labels = []






for folder in os.listdir("dataset/train/"):
    print(folder)
    
    for file in os.listdir("dataset/train/" + folder + "/"):
        print(file)
        name = 'dataset/train/'+ folder +"/" + file
        print(name)
        train_img = cv2.imread('dataset/train/'+ folder +"/" + file)
        if train_img is None:
            print("no image")
        cv2.imshow("imgt",train_img)
        cv2.waitKey(0)

        
        train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)

        
        Keypoints, Descriptors = orb.detectAndCompute(train_img_bw,None)
        print(Descriptors)
        trainDescriptors.append(Descriptors.flatten())
        trainKeyposints.append(Keypoints)
        labels.append(int(folder))
        print(len(Descriptors.flatten()))

00004
01186_00002.ppm
dataset/train/00004/01186_00002.ppm


NameError: name 'orb' is not defined

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier

# #Create KNN Classifier
# knn = KNeighborsClassifier(n_neighbors=5)

# #Train the model using the training sets
# knn.fit(trainDescriptors, labels)

In [ ]:
from sklearn import svm
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(trainDescriptors, labels)

In [7]:

query_img = cv2.imread('dataset/test/00004/00719_00002.ppm')
  


    
# Convert it to grayscale
query_img_bw = cv2.cvtColor(query_img,cv2.COLOR_BGR2GRAY)

  
# Initialize the ORB detector algorithm
orb = cv2.ORB_create()
  
# Now detect the keypoints and compute
# the descriptors for the query image
# and train image
queryKeypoints, queryDescriptors = orb.detectAndCompute(query_img_bw,None)

print(len(queryDescriptors))


clf.predict([queryDescriptors.flatten()])

49


NameError: name 'clf' is not defined

In [ ]:
len(queryDescriptors)

In [ ]:
for i in range(0,len(trainDescriptors)):
    print(len(trainDescriptors[i]))